In [107]:
import subprocess
import json

result = subprocess.run([
    'target/debug/scorer-playground',
    # Add positive and negative ids here
    '--positives', '4221', 
    '--negatives', '4177,2897'
    ], stdout=subprocess.PIPE).stdout.decode('utf-8')

scored_points = json.loads(result)
print(scored_points)

[{'id': 10136, 'version': 486, 'score': 0.055109024, 'payload': {'cafe': {'address': 'Kopenhagener Str. 12A', 'location': {'lat': 52.54903442589229, 'lon': 13.409768305558718}, 'menu_id': '5f0884e60b8f78538dc16610', 'name': 'Estelle Dining', 'rating': 9.2, 'slug': 'estelle-dining', 'website': 'https://estelle-dining.com'}, 'description': 'vegan\nwith pumpkin, kale and pumpkin seed salsa', 'image': 'https://wolt-menu-images-cdn.wolt.com/menu-images/5f0884e60b8f78538dc16610/19382e2a-137e-11eb-aec1-5e9d4e986ba7_image0.jpeg', 'image_file': '19382e2a-137e-11eb-aec1-5e9d4e986ba7_image0.jpeg', 'name': 'Pumpkin Pizza'}, 'vector': None}, {'id': 13227, 'version': 625, 'score': 0.046364903, 'payload': {'cafe': {'address': 'Gabriele-Tergit-Promenade 21 - Entrance : Köthener Str.', 'location': {'lat': 52.503923, 'lon': 13.3747721}, 'menu_id': '5fd72bb5663152b79fb2a36a', 'name': 'Pinch', 'rating': 9.6, 'slug': 'pinch', 'website': 'https://pinch-cafe.business.site/'}, 'description': 'mit Ingwer, Koko

In [108]:
from IPython.display import Image

for point in scored_points:
    print(point["id"])
    image = (point["payload"]['image'])
    display(Image(url= image, width=300))

10136


13227


2777


11570


3024


6643


146


10329


6041


8659
